In [1]:
from circuit import gen_random_circuits
from circuit.formatter import qiskit_to_my_format_circuit, layered_circuits_to_qiskit, get_layered_instructions
from upstream import RandomwalkModel
from collections import defaultdict

import math
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend, gen_linear_topology, get_linear_neighbor_info, gen_fulllyconnected_topology

from qiskit import QuantumCircuit
import pennylane as qml
import numpy as np
from downstream.synthesis.pennylane_op_jax import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op_jax import layer_circuit_to_matrix

from scipy.stats import unitary_group

from downstream.synthesis.synthesis_model_pca_unitary_jax import find_parmas, pkl_dump, pkl_load, matrix_distance_squared, SynthesisModel, synthesize
from itertools import combinations
import time
from qiskit import transpile
import random
import cloudpickle as pickle
from circuit.formatter import layered_circuits_to_qiskit, to_unitary
from qiskit.quantum_info import Operator
from jax import vmap
from utils.unitaries import qft_U, grover_U


In [7]:

n_qubits = 3
topology = gen_fulllyconnected_topology(n_qubits)
neigh_info = gen_fulllyconnected_topology(n_qubits)

# topology = gen_linear_topology(n_qubits)
# neigh_info = get_linear_neighbor_info(n_qubits, 1)
synthesis_data_path = f'./temp_data/{n_qubits}_synthesis_data.pkl'
regen = True
if regen:
    backend = Backend(n_qubits=n_qubits, topology=topology, neighbor_info=neigh_info, basis_single_gates=['u'],
                    basis_two_gates=['cz'], divide=False, decoupling=False)


    min_gate, max_gate = 2 * 4**n_qubits - 200, 2 * 4**n_qubits
    dataset = gen_random_circuits(min_gate=100, max_gate=max_gate, gate_num_step=max_gate//50, n_circuits=50,
                                    two_qubit_gate_probs=[2, 5], backend=backend, reverse=False, optimize=True, multi_process=True)


    upstream_model = RandomwalkModel(1, 20, backend)
    upstream_model.train(dataset, multi_process=True, remove_redundancy=False)
    synthesis_model = SynthesisModel(upstream_model, f'synthesis_{n_qubits}')
    data = synthesis_model.construct_data(dataset, multi_process = True)
    synthesis_model.construct_model(data)
    synthesis_model.save()
    
    with open(synthesis_data_path, 'wb') as f:
        pickle.dump(data, f)
        
    Us, Vs = data
else:
    synthesis_model: SynthesisModel = SynthesisModel.load(f'synthesis_{n_qubits}')
    backend: Backend = synthesis_model.backend

    with open(synthesis_data_path, 'rb') as f:
        Us, Vs = pickle.load(f)

(_gen_random_circuits_remote pid=3616) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: divide by zero encountered in det
(_gen_random_circuits_remote pid=3616)   r = _umath_linalg.det(a, signature=signature)
(_gen_random_circuits_remote pid=3616) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: invalid value encountered in det
(_gen_random_circuits_remote pid=3616)   r = _umath_linalg.det(a, signature=signature)
(_gen_random_circuits_remote pid=3622) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: divide by zero encountered in det
(_gen_random_circuits_remote pid=3622)   r = _umath_linalg.det(a, signature=signature)
(_gen_random_circuits_remote pid=3622) /Users/siwei/miniforge3/envs/tf2/lib/python3.8/site-packages/numpy/linalg/linalg.py:2147: RuntimeWarning: invalid value encountered in det
(_gen_random_circuits_rem

finish random circuit generation with 2100 circuits
(remote_train pid=3622) train:0/210, 420th offest
(remote_train pid=3616) train:0/210, 1050th offest
(remote_train pid=3615) train:0/210, 1260th offest
(remote_train pid=3617) train:0/210, 630th offest
(remote_train pid=3621) train:0/210, 210th offest
(remote_train pid=3614) train:0/210, 1470th offest
(remote_train pid=3612) train:0/210, 840th offest
(remote_train pid=3613) train:0/210, 0th offest
(remote_train pid=3619) train:0/210, 1680th offest
(remote_train pid=3618) train:0/210, 1890th offest
(remote_train pid=3613) train:100/210, 0th offest
(remote_train pid=3616) train:100/210, 1050th offest
(remote_train pid=3617) train:100/210, 630th offest
(remote_train pid=3621) train:100/210, 210th offest
(remote_train pid=3614) train:100/210, 1470th offest
(remote_train pid=3619) train:100/210, 1680th offest
(remote_train pid=3612) train:100/210, 840th offest
(remote_train pid=3613) train:200/210, 0th offest
(remote_train pid=3622) train:

100%|██████████| 2100/2100 [02:33<00:00, 13.65it/s]
/Users/siwei/workspace/QuCT-Micro2023/downstream/synthesis/synthesis_model_pca_unitary_jax.py:269: ComplexWarning: Casting complex values to real discards the imaginary part
  Us = np.array(Us, dtype=np.float64)  # [:100000]


len(Us) =  455930 len(gate_vecs) =  455930
Start construct model
k = 6
reduced_prop = (0.7520537288813981+0j)
Finish construct model, costing 15.18346905708313s


In [32]:
U = unitary_group.rvs(2**n_qubits)
dists = vmap(matrix_distance_squared, in_axes = (None, 0))(U, Us)
np.argmin(dists), np.min(dists)


(Array(281922, dtype=int64), Array(0.66949445, dtype=float64))

In [23]:
matrix_distance_squared(unitary_group.rvs(2**n_qubits), unitary_group.rvs(2**n_qubits))

Array(0.87738657, dtype=float64)

In [33]:
dists.size

455930

In [55]:
_n_qubits = 5
_Us = np.array([unitary_group.rvs(2**_n_qubits) for i in range(500000)])

In [56]:
U = unitary_group.rvs(2**_n_qubits)
dists = vmap(matrix_distance_squared, in_axes = (None, 0))(U, _Us)
np.argmin(dists), np.min(dists)

(Array(359020, dtype=int64), Array(0.86629948, dtype=float64))

In [54]:
U.shape, _Us.shape

((32, 32), (100000, 32, 32))